<a href="https://colab.research.google.com/github/alyrazik/CUDA/blob/main/CUDA_code_coursework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s19rx3b2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s19rx3b2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=bbd6a1c17f413e057155ce6507dcc398aa0a29865e478919538ac02079070354
  Stored in directory: /tmp/pip-ephem-wheel-cache-jxan__17/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
 %reload_ext nvcc_plugin

#Below code implements three functions; two implementation for the GPU and one for the CPU. 

In [106]:
%%cu 
#include <iostream> 
#include <stdlib.h>
#include <cstdio>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <chrono>
#include <iomanip>
using namespace std; 



// __global__ means this function will be executed on the GPU and is callable from the host.
// this version of the function parallizes the outer loop only.

__global__ void GTensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{

  int i = threadIdx.x; // i is unique for each thread.
 
if (i<n) 
// so we won't run into a problem in case the number of threads exceeded the...
// array elements N. (we'll think of a better way to parllelize the second loop
// as well and them make SIZE n*m
   for (int j=0; j<m; j++) 
   {
       d[i*m+j]=a[i]*b[j]+c[i*m+j];
   }
      
} 

// __global__ means this function will be executed on the GPU and is callable from the host.
// this version of the function parallizes the inner loop as well as the outer loop.

__global__ void G2TensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{

  int i = threadIdx.x; // i is unique for each thread.
 
if (i<n*m) 
// so we won't run into a problem in case the number of threads exceeded the...
// array elements N*M. 
      //i/M counts one (increments) every M elements until it reaches the count of N.
      //i%M counts M times (from 0 to M-1) and then repeats this series N times.
      //this basically accomplishes the multiplication process between the two vectors
      d[i]=a[i/m]*b[i%m]+c[i]; 
}
// and the CPU implementation.

void TensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{
 for (int i=0; i<n; i++)
   for (int j=0; j<m; j++) 
   {
       d[i*m+j]=a[i]*b[j]+c[i*m+j];
   }
      
}




int main() 
{ 
    
// Generate a, b, c matrices with random elements using N and M
// they will be used to initalize the variables for both CPU and GPU.
cout<<"N       M       GPU1    GPU(NxM)  CPU"<<endl;
int N;
int M=10;
for (N =10; N<=10000; N=10*N)
{
//cout<<"\nLooping with M="<<M<<" and N="<<N<<endl;
cout<<N<<"\t"<<M<<"\t";

float* A = new float[N];
float* B = new float[M];
float* C = new float[N*M];

for(int i=0;i<N;i++)
      A[i]=rand()%100;
for(int i=0;i<M;i++)
      B[i]=rand()%100;
for(int i=0;i<N*M;i++)
      C[i]=rand()%100;


//cout << "This code runs the operation on GPU\n";
int n=N;
int m=M;

float*a1, *b1, *c1, *d1;
//allocate memory that is shared between the GPU device and the host CPU.
cudaMallocManaged(&a1, n*sizeof(float)); 
cudaMallocManaged(&b1, m*sizeof(float));
cudaMallocManaged(&c1, n*m*sizeof(float));
cudaMallocManaged(&d1, n*m*sizeof(float));

//initialize matrices
for(int i=0;i<N;i++)
      a1[i]=A[i];
for(int i=0;i<M;i++)
      b1[i]=B[i];
for(int i=0;i<N*M;i++)
      c1[i]=C[i];


//to calculate the time
cudaEvent_t start, end;
cudaEventCreate(&start); 
cudaEventCreate(&end); 
cudaEventRecord(start);

//Kernel launch
 
GTensorsOperation<<<1, N>>>(a1, b1, c1, d1, n, m); 

//1 is no. of thread blocks
//3, the second p/m is the number of threads within each block. usually the number of elements in a vector, the number of iterations.

cudaEventRecord(end); 
//get the CPU to wait for all kernels to finish.
//cudaDeviceSynchronize(); //do I need it?
cudaEventSynchronize(end);  

float time = 0; 
cudaEventElapsedTime(&time, start, end);  
//cout<<"Time taken "
cout<<setprecision(2)<<time<<"\t";//"    ";


//Try the next function  G2TensorsOperation<<<1, N>>>(a, b, c, d, n, m); 
 
//cout << "\nThis code runs the operation on GPU using N*M threads.\n";
cudaEventRecord(start);
G2TensorsOperation<<<1, N*M>>>(a1, b1, c1, d1, n, m); 
cudaEventRecord(end); 
cudaEventSynchronize(end); 
time = 0;
cudaEventElapsedTime(&time, start, end);  
// cudaEventElapsedTime computes elapsed time between two events (in milliseconds with a resolution of around 0.5 microseconds)
// source: docs.nvidia.com
//cout<<"Time taken "<<
cout<<setprecision(2)<<time<<"\t  ";//"    ";
 

//free the allocated memory
cudaFree(a1);
cudaFree(b1);
cudaFree(c1);
cudaFree(d1);


//now the CPU version:

//cout << "\nThis code runs the operation on CPU\n";
float* a = new float[n];
float* b = new float[m];
float* c = new float[n*m];
float* d = new float[n*m];

//initialize matrices
for(int i=0;i<N;i++)
      a[i]=A[i];
for(int i=0;i<M;i++)
      b[i]=B[i];
for(int i=0;i<N*M;i++)
      c[i]=C[i];

//calculate time
using chrono::high_resolution_clock;
using chrono::duration_cast;
using chrono::duration;
using chrono::milliseconds;
auto t_start = high_resolution_clock::now();

 TensorsOperation(a, b, c, d, n, m);
auto t_end = high_resolution_clock::now();

duration<double, std::milli> ms_double = t_end - t_start;
//cout << "Time taken: " 
cout<<setprecision(4)<< ms_double.count() <<endl;

/*
cout<<"Result matrix D: \n" ;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<d[i*m+j]<<" ";
  cout<<endl;
 }
*/
//cout<<endl;
M=M*10; 

delete a, b, c, d;

}


return 0; 
} 


N       M       GPU1    GPU(NxM)  CPU
10	10	0.18	0.0093	  0.000678
100	100	0.23	0.0023	  0.04686
1000	1000	4.5	0.003	  3.763
10000	10000	0.0038	0.0024	  489.5



#Below code displays the contents of d matrix to investigate accuracy

In [96]:
%%cu 
#include <iostream> 
#include <stdlib.h>
#include <cstdio>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <chrono>
#include <iomanip>
using namespace std; 

#define N 1000
#define M 1000


// __global__ means this function will be executed on the GPU and is callable from the host.
// this version of the function parallizes the outer loop only.

__global__ void GTensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{

  int i = threadIdx.x; // i is unique for each thread.
 
if (i<N) 
// so we won't run into a problem in case the number of threads exceeded the...
// array elements N. (we'll think of a better way to parllelize the second loop
// as well and them make SIZE n*m
   for (int j=0; j<m; j++) 
   {
       d[i*m+j]=a[i]*b[j]+c[i*m+j];
   }
      
} 

// __global__ means this function will be executed on the GPU and is callable from the host.
// this version of the function parallizes the inner loop as well as the outer loop.

__global__ void G2TensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{

  int i = threadIdx.x; // i is unique for each thread.
 
if (i<N*M) 
// so we won't run into a problem in case the number of threads exceeded the...
// array elements N*M. 
      //i/M counts one (increments) every M elements until it reaches the count of N.
      //i%M counts M times (from 0 to M-1) and then repeats this series N times.
      //this basically accomplishes the multiplication process between the two vectors
      d[i]=a[i/M]*b[i%M]+c[i]; 
}
// and the CPU implementation.

void TensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{
 for (int i=0; i<n; i++)
   for (int j=0; j<m; j++) 
   {
       d[i*m+j]=a[i]*b[j]+c[i*m+j];
   }
      
}




int main() 
{ 
    
// Generate a, b, c matrices with random elements using N and M
// they will be used to initalize the variables for both CPU and GPU.

float* A = new float[N];
float* B = new float[M];
float* C = new float[N*M];

for(int i=0;i<N;i++)
      A[i]=rand()%10/1000000000.0f;
for(int i=0;i<M;i++)
      B[i]=rand()%10/1000000000.0f;
for(int i=0;i<N*M;i++)
      C[i]=rand()%10/1000000000.0f; //10000000.0f;


// Running first function on GPU
cout << "This code runs the operation on GPU\n";
int n=N;
int m=M;

float*a1, *b1, *c1, *d1;
//allocate memory that is shared between the GPU device and the host CPU.
cudaMallocManaged(&a1, n*sizeof(float)); 
cudaMallocManaged(&b1, m*sizeof(float));
cudaMallocManaged(&c1, n*m*sizeof(float));
cudaMallocManaged(&d1, n*m*sizeof(float));

//initialize matrices
for(int i=0;i<N;i++)
      a1[i]=A[i];
for(int i=0;i<M;i++)
      b1[i]=B[i];
for(int i=0;i<N*M;i++)
      c1[i]=C[i];

//to calculate the time
cudaEvent_t start, end;
cudaEventCreate(&start); 
cudaEventCreate(&end); 
 
cudaEventRecord(start);

//Kernel launch
 
GTensorsOperation<<<1, N>>>(a1, b1, c1, d1, n, m); 

//1 is no. of thread blocks
//3, the second p/m is the number of threads within each block. usually the number of elements in a vector, the number of iterations.

cudaEventRecord(end); 
 
//get the CPU to wait for all kernels to finish.
//cudaDeviceSynchronize(); //do I need it?
cudaEventSynchronize(end);  

float time = 0; 
cudaEventElapsedTime(&time, start, end);  
cout<<"Time taken "<<time<<endl;
if(N<11)
{
cout<<"Result matrix D: "<<endl;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<setprecision(15)<<d1[i*m+j]<<"\t";
  cout<<endl;
 }
}
 


///*
//Try the next function  G2TensorsOperation<<<1, N>>>(a, b, c, d, n, m); 
float*a2, *b2, *c2, *d2;
//allocate memory that is shared between the GPU device and the host CPU.
cudaMallocManaged(&a2, n*sizeof(float)); 
cudaMallocManaged(&b2, m*sizeof(float));
cudaMallocManaged(&c2, n*m*sizeof(float));
cudaMallocManaged(&d2, n*m*sizeof(float));

//initialize matrices
for(int i=0;i<N;i++)
      a2[i]=A[i];
for(int i=0;i<M;i++)
      b2[i]=B[i];
for(int i=0;i<N*M;i++)
      c2[i]=C[i];

cout << "\nThis code runs the operation on GPU using N*M threads.\n";
cudaEventRecord(start);
G2TensorsOperation<<<1, N*M>>>(a2, b2, c2, d2, n, m); 
cudaEventRecord(end); 
cudaEventSynchronize(end); 
time = 0;
cudaEventElapsedTime(&time, start, end);  
 
// cudaEventElapsedTime computes elapsed time between two events (in milliseconds with a resolution of around 0.5 microseconds)
// source: docs.nvidia.com
 
cout<<"Time taken "<<time<<endl;
 
if(N<11)
{
cout<<"Result matrix D: "<<endl;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<setprecision(15)<<d2[i*m+j]<<"\t";
  cout<<endl;
 }
}




//now the CPU version:

cout << "\nThis code runs the operation on CPU\n";
float* a = new float[n];
float* b = new float[m];
float* c = new float[n*m];
float* d = new float[n*m];

//initialize matrices
for(int i=0;i<N;i++)
      a[i]=A[i];
for(int i=0;i<M;i++)
      b[i]=B[i];
for(int i=0;i<N*M;i++)
      c[i]=C[i];

//calculate time
using chrono::high_resolution_clock;
using chrono::duration_cast;
using chrono::duration;
using chrono::milliseconds;
auto t_start = high_resolution_clock::now();
TensorsOperation(a, b, c, d, n, m);
auto t_end = high_resolution_clock::now();

duration<double, std::milli> ms_double = t_end - t_start;
cout << "Time taken: " << ms_double.count() << "ms"<<endl;

if(N<11)
{
cout<<"Result matrix D: \n" ;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<setprecision(15)<<d[i*m+j]<<"\t";
  cout<<endl;
 }
}
 
cout<<"\nError between CPU and GPU \n" ;
double error =0.0;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    //cout<<(d[i*m+j])<<" ";
    //printf("%.9lg ", d[i*m+j]);
    error+=abs(d[i*m+j]-d2[i*m+j]);
 }
 cout<<setprecision(15)<<error<<endl;

//free the allocated memory
cudaFree(a1);
cudaFree(b1);
cudaFree(c1);
cudaFree(d1);
cudaFree(a2);
cudaFree(b2);
cudaFree(c2);
cudaFree(d2);
delete a, b, c, d;
 
return 0; 
} 


This code runs the operation on GPU
Time taken 4.48115

This code runs the operation on GPU using N*M threads.
Time taken 0.002624

This code runs the operation on CPU
Time taken: 3.62443ms

Error between CPU and GPU 
0.00450293696432537



In [ ]:
!nvidia-smi

In [49]:
// this code runs math functions on both CPU and GPU to evidently compare accuracies. 
%%cu 
#include <cuda.h>
#include <stdio.h>
#include <iomanip>
#include <iostream>
using namespace std;

__global__ void calcGPU() {
	float x = 0.000001f;
	float y = expf(x);
	printf("GPU: %.9lg\n", y);
  //cout<<setprecision(15)<<y;
}

void calcCPU() {
	float x = 0.000001f;
	float y = expf(x);
	//printf("CPU: %.9lg\n", y);
  cout<<setprecision(15)<<y;
}

int main() {

	calcGPU<<<1, 1>>>();
	cudaDeviceReset();
	calcCPU();
	return 0;
}

GPU: 1.00000107
1.00000095367432
